In [149]:
# the results
import plotly.express as px
import pandas as pd
import statsmodels.api as sm

# Load data and create scatter plot with trendline
df = pd.read_csv("funding_only.csv")
fig = px.scatter(
    df,
    x="Amount Raised",
    y="plr_norm",
    hover_name="Company",
    custom_data=["timestamped_link"],  # Add URL data for click functionality
    trendline="ols",
    title=""
)

# Style trendline and markers in single updates
fig.update_traces(
    selector=dict(mode="lines"),
    line=dict(color="black", width=2, dash="dot"),
    name="OLS Fit",
    hoverinfo="skip",
    hovertemplate=None
)

fig.update_traces(
    selector=dict(mode="markers"),
    marker=dict(color="black", size=5, opacity=.8, line=dict(width=0)),
    hovertemplate="<span style='font-weight: 600; font-family: gill sans;'>%{hovertext}</span><br>$%{x:.1f}M<br>PLR: %{y:.2f}<extra></extra>",
    hoverlabel=dict(
        font=dict(family="gill sans", color="black"),
        bgcolor="white",
        bordercolor="black"
    )
)

# Calculate R² for annotation
X = sm.add_constant(df['Amount Raised'])
r2 = sm.OLS(df['plr_norm'], X).fit().rsquared

# Complete layout configuration
fig.update_layout(
    template="simple_white",
    font=dict(family="monotype bembo", size=14, color="black"),
    xaxis_title="Funding ($M)",
    yaxis_title="Loudness",
    showlegend=False,
    width=647.2,
    height=400,
    margin=dict(l=40, r=20, t=40, b=40),
    title_font=dict(size=18),
    dragmode=False,
    xaxis=dict(
        range=[0, 300],
        showgrid=False,
        showline=True,
        zeroline=False,
        tickfont=dict(size=14),
        fixedrange=True
    ),
    yaxis=dict(
        range=[.2, 1],
        tickmode='array',
        tickvals=[0.2, 0.4, 0.6, 0.8, 1],
        ticktext=['.2', '.4', '.6', '.8', '1'],
        showgrid=False,
        showline=True,
        zeroline=False,
        tickfont=dict(size=14),
        fixedrange=True
    )
)

# Add R² annotation
fig.add_annotation(
    xref="paper", yref="paper",
    x=0.98, y=0.92,
    text=f"R² = {r2:.2f}",
    showarrow=False,
    font=dict(size=12, family="gill sans", color="black")
)

# Show chart with click functionality enabled
config = {
    "displaylogo": False, 
    "displayModeBar": False
}
fig.show(config=config)

# Write HTML with embedded JavaScript for click handling
html_string = fig.to_html(config=config, include_plotlyjs=True)

# Add JavaScript for click handling
click_js = '''
<script>
document.addEventListener('DOMContentLoaded', function() {
    setTimeout(function() {
        var plotDiv = document.querySelector('.js-plotly-plot');
        if (plotDiv) {
            plotDiv.on('plotly_click', function(data) {
                if (data.points && data.points[0] && data.points[0].customdata) {
                    var url = data.points[0].customdata[0];
                    if (url) {
                        window.open(url, '_blank');
                    }
                }
            });
        }
    }, 100);
});
</script>
'''

# Insert JavaScript before closing body tag
html_with_clicks = html_string.replace('</body>', click_js + '</body>')

# Save the interactive HTML file
with open("chart_1_the_results.html", "w") as f:
    f.write(html_with_clicks)


In [121]:
# John versus Jordi loudness boxplot
import plotly.express as px
import pandas as pd
import numpy as np

# Load funding_only.csv
df = pd.read_csv("funding_only.csv")

# Filter for John and Jordi only
df_filtered = df[df["Gong Master"].isin(["John", "Jordi"])]

# Create minimalist boxplot with color separation
fig = px.box(
    df_filtered,
    x="Gong Master",
    y="plr_norm",
    points="all",
    color="Gong Master",  # This creates separate traces for each person
    category_orders={"Gong Master": ["John", "Jordi"]},
    #title="Jordi is louder than John"
)

# Minimalist styling
fig.update_layout(
    template="simple_white",
    font=dict(family="Times New Roman", size=14, color="black"),
    xaxis_title="",
    yaxis_title="Loudness",
    showlegend=False,
    width=404.5,  # Smaller, more compact
    height=500,
    margin=dict(l=60, r=20, t=40, b=40),  # Minimal margins
    plot_bgcolor="white",
    paper_bgcolor="white"
)

# Update axis properties separately to ensure font consistency
fig.update_xaxes(
    showgrid=False,
    showline=False,  # Remove axis line for minimalism
    zeroline=False,
    tickfont=dict(size=14, family="Times New Roman", color="black")
)

fig.update_yaxes(
    range=[.2, 1],
    tickmode='array',
    tickvals=[.2, .4, .6, .8, 1],
    showgrid=False,
    showline=True,
    linecolor="white",
    linewidth=1,
    zeroline=False,
    tickfont=dict(size=14, family="Times New Roman", color="black")
)

# Force x-axis category labels to match y-axis font size - try multiple approaches
fig.update_layout(
    xaxis=dict(tickfont=dict(size=14, family="Times New Roman", color="black")),
    font=dict(size=14, family="Times New Roman", color="black")  # Override global font
)

# Minimalist box styling with different colors for each person
#colors = {"John": "#009966", "Jordi": "#ffe020"}
colors = {"John": "white", "Jordi": "white"}
for trace in fig.data:
    # Extract the gong master name from the trace name
    gong_master = trace.name
    trace.update(
        fillcolor=colors.get(gong_master, "white"),
        line=dict(color="black", width=.75),
        marker=dict(
            color="black",
            size=4,
            opacity=0.7
        ),
        boxpoints="all",  # Only show outliers
        hoverinfo="none"  # Disable hover completely
    )

# Add value labels for each boxplot
for i, gong_master in enumerate(["John", "Jordi"]):
    data_subset = df_filtered[df_filtered["Gong Master"] == gong_master]["plr_norm"]
    
    # Calculate statistics
    q1 = data_subset.quantile(0.25)
    median = data_subset.median()
    q3 = data_subset.quantile(0.75)
    min_val = data_subset.min()
    max_val = data_subset.max()
    
    # Add annotations for key values
    x_pos = i  # 0 for John, 1 for Jordi
    offset = 0.275  # Horizontal offset from center (increase for more spacing)
    
    # Median label
    fig.add_annotation(
        x=x_pos + offset, y=median,
        text=f"{median:.2f}",
        showarrow=False,
        font=dict(family="gill sans", size=12, color="black"),
        xanchor="left"
    )
    
    # Q1 label
    #fig.add_annotation(
    #    x=x_pos + offset, y=q1,
    #    text=f"{q1:.2f}",
    #    showarrow=False,
    #    font=dict(family="gill sans", size=12, color="black"),
    #    xanchor="left"
    #)
    
    # Q3 label
    #fig.add_annotation(
    #    x=x_pos + offset, y=q3,
    #    text=f"{q3:.2f}",
    #    showarrow=False,
    #    font=dict(family="gill sans", size=12, color="black"),
    #    xanchor="left"
    #)

# Static chart config
config = {
    "displaylogo": False,
    "displayModeBar": False,  # Hide toolbar
    "staticPlot": True  # Make it completely static
}

fig.show(config=config)

#fig.write_html("chart_2_jordi_john_boxplot.html")

In [77]:
# John versus Jordi amount raised boxplot
import plotly.express as px
import pandas as pd
import numpy as np

# Load funding_only.csv
df = pd.read_csv("funding_only.csv")

# Filter for John and Jordi only
df_filtered = df[df["Gong Master"].isin(["John", "Jordi"])]

# Create minimalist boxplot with color separation
fig = px.box(
    df_filtered,
    x="Gong Master",
    y="Amount Raised",
    color="Gong Master",  # This creates separate traces for each person
    category_orders={"Gong Master": ["John", "Jordi"]},
    #title="Jordi vs John Amount Raised"
)

# Minimalist styling
fig.update_layout(
    template="simple_white",
    font=dict(family="Times New Roman", size=14, color="black"),
    xaxis_title="",
    yaxis_title="Amount Raised ($M)",
    showlegend=False,
    width=300,  # Smaller, more compact
    height=400,
    margin=dict(l=60, r=20, t=40, b=40),  # Minimal margins
    plot_bgcolor="white",
    paper_bgcolor="white"
)

# Update axis properties separately to ensure font consistency
fig.update_xaxes(
    showgrid=False,
    showline=False,  # Remove axis line for minimalism
    zeroline=False,
    tickfont=dict(size=14, family="Times New Roman", color="black")
)

fig.update_yaxes(
    showgrid=False,
    showline=True,
    linecolor="white",
    linewidth=1,
    zeroline=False,
    tickfont=dict(size=14, family="Times New Roman", color="black")
)

# Force x-axis category labels to match y-axis font size - try multiple approaches
fig.update_layout(
    xaxis=dict(tickfont=dict(size=14, family="Times New Roman", color="black")),
    font=dict(size=14, family="Times New Roman", color="black")  # Override global font
)

# Minimalist box styling with different colors for each person
colors = {"John": "white", "Jordi": "white"}
for trace in fig.data:
    # Extract the gong master name from the trace name
    gong_master = trace.name
    trace.update(
        fillcolor=colors.get(gong_master, "white"),
        line=dict(color="black", width=.75),
        marker=dict(
            color="black",
            size=3,
            opacity=0.6
        ),
        boxpoints="outliers",  # Only show outliers
        hoverinfo="none"  # Disable hover completely
    )

# Add value labels for each boxplot
for i, gong_master in enumerate(["John", "Jordi"]):
    data_subset = df_filtered[df_filtered["Gong Master"] == gong_master]["Amount Raised"]
    
    # Calculate statistics
    q1 = data_subset.quantile(0.25)
    median = data_subset.median()
    q3 = data_subset.quantile(0.75)
    min_val = data_subset.min()
    max_val = data_subset.max()
    
    # Add annotations for key values
    x_pos = i  # 0 for John, 1 for Jordi
    offset = 0.25  # Horizontal offset from center (increase for more spacing)
    
    # Median label
    fig.add_annotation(
        x=x_pos + offset, y=median,
        text=f"${median:.1f}M",
        showarrow=False,
        font=dict(family="Times New Roman", size=10, color="black"),
        xanchor="left"
    )

# Static chart config
config = {
    "displaylogo": False,
    "displayModeBar": False,  # Hide toolbar
    "staticPlot": True  # Make it completely static
}

fig.show(config=config)

fig.write_html("chart_3_jordi_john_amount_raised_boxplot.html")


In [46]:
# John vs Jordi gong hits by Funding Stage
import pandas as pd
import plotly.graph_objects as go

# Load data and count by funding stage
df = pd.read_csv("funding_only.csv")
stages = ["Seed", "Series A", "Series B", "Series C", "Series D or Later"]

# Count data for John and Jordi
john_counts = df[df['Gong Master'] == 'John']['Funding Stage'].value_counts()
jordi_counts = df[df['Gong Master'] == 'Jordi']['Funding Stage'].value_counts()

# Create figure with predefined colors
colors = {"John": "#009966", "Jordi": "#ffe020"}
fig = go.Figure()

# Add bars with final styling
for name, counts, offset in [("John", john_counts, 0), ("Jordi", jordi_counts, 1)]:
    fig.add_trace(go.Bar(
        name=name,
        x=stages,
        y=[counts.get(stage, 0) for stage in stages],
        marker=dict(
            color=colors[name],
            line=dict(width=0.5, color="black"),
            opacity=0.8
        ),
        offsetgroup=offset,
        hoverinfo="none"
    ))

# Layout configuration
fig.update_layout(
    template="simple_white",
    font=dict(family="Times New Roman", size=12, color="black"),
    xaxis_title="",
    yaxis_title="Value",
    showlegend=True,
    width=600,
    height=400,
    margin=dict(l=40, r=20, t=50, b=60),
    xaxis=dict(
        showgrid=False,
        showline=True,
        linecolor="black",
        linewidth=1,
        zeroline=False,
        tickfont=dict(size=10),
        tickangle=-45
    ),
    yaxis=dict(
        range=[0, 20],
        tickmode='array',
        tickvals=list(range(0, 21, 2)),
        showgrid=False,
        showline=True,
        linecolor="black",
        linewidth=1,
        zeroline=False,
        tickfont=dict(size=12)
    ),
    plot_bgcolor="white",
    paper_bgcolor="white",
    legend=dict(
        x=0.65,
        y=0.9,
        bgcolor="rgba(255,255,255,0)",
        bordercolor="rgba(0,0,0,0)",
        font=dict(family="Times New Roman", size=10, color="black")
    )
)

# Show chart
fig.show(config={"displaylogo": False, "displayModeBar": False, "staticPlot": True})

fig.write_html("chart_4_gong_hits_by_funding_stage.html")

In [48]:
# John has done more sitting Series A gong hits than Jordi has ever done
import plotly.express as px
import pandas as pd
import numpy as np

# Load funding_only.csv
df = pd.read_csv("funding_only.csv")

# Create separate DataFrames for John and Jordi
john_df = df[df['Gong Master'] == 'John'].copy()
john_df['John_Status'] = john_df['john_sitting'].apply(lambda x: 'Sitting' if x == 1 else 'Standing')

jordi_df = df[df['Gong Master'] == 'Jordi'].copy()

# Create subplot with grouped bars for John and separate bars for Jordi
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Count data for each category
john_standing_counts = john_df[john_df['John_Status'] == 'Standing']['Funding Stage'].value_counts()
john_sitting_counts = john_df[john_df['John_Status'] == 'Sitting']['Funding Stage'].value_counts()
jordi_counts = jordi_df['Funding Stage'].value_counts()

# Define stages in order
stages = ["Seed", "Series A", "Series B", "Series C", "Series D or Later"]

# Create figure
fig = go.Figure()

# Add John Sitting bars (base layer)
fig.add_trace(go.Bar(
    name='John (Sitting)',
    x=stages,
    y=[john_sitting_counts.get(stage, 0) for stage in stages],
    marker_color='#90EE90',
    offsetgroup=0,  # John's group
    base=0  # Start from bottom
))

# Add John Standing bars (stacked on top)
fig.add_trace(go.Bar(
    name='John (Standing)',
    x=stages,
    y=[john_standing_counts.get(stage, 0) for stage in stages],
    marker_color='#009966',
    offsetgroup=0,  # Same group as John Sitting
    base=[john_sitting_counts.get(stage, 0) for stage in stages]  # Stack on top of sitting
))

# Add Jordi bars (separate group)
fig.add_trace(go.Bar(
    name='Jordi',
    x=stages,
    y=[jordi_counts.get(stage, 0) for stage in stages],
    marker_color='#ffe020',
    offsetgroup=1  # Separate group from John
))

# Minimalist styling
fig.update_layout(
    template="simple_white",
    font=dict(family="Times New Roman", size=12, color="black"),
    xaxis_title="",  # Remove x-axis title for minimalism
    yaxis_title="Value",
    showlegend=True,
    width=600,  # Slightly wider for 3 categories
    height=400,
    margin=dict(l=40, r=20, t=20, b=60),  # Extra bottom margin for stage labels
    xaxis=dict(
        showgrid=False,
        showline=True,
        linecolor="black",
        linewidth=1,
        zeroline=False,
        tickfont=dict(size=10),
        tickangle=-45  # Angle the stage labels
    ),
    yaxis=dict(
        range=[0, 20],
        tickmode='array',
        tickvals=[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
        showgrid=False,
        showline=True,
        linecolor="black",
        linewidth=1,
        zeroline=False,
        tickfont=dict(size=12)
    ),
    plot_bgcolor="white",
    paper_bgcolor="white",
    legend=dict(
        x=0.65,
        y=0.9,
        bgcolor="rgba(255,255,255,0)",  # Transparent legend background
        bordercolor="rgba(0,0,0,0)",
        font=dict(family="Times New Roman", size=10, color="black")
    )
)

# Update bar colors and styling
fig.update_traces(
    marker=dict(
        line=dict(width=0.5, color="black"),
        opacity=0.8
    ),
    hoverinfo="none"  # Disable hover
)

# Set specific colors for John standing, John sitting, and Jordi
colors = {
    "John (Standing)": "#009966",  # Original John green
    "John (Sitting)": "#90EE90",   # Light green for sitting
    "Jordi": "#ffe020"             # Brown for Jordi
}
for trace in fig.data:
    if trace.name in colors:
        trace.marker.color = colors[trace.name]

# Static chart config
config = {
    "displaylogo": False,
    "displayModeBar": False,  # Hide toolbar
    "staticPlot": True  # Make it completely static
}

fig.show(config=config)

fig.write_html("chart_5_sitting_vs_jordi.html")

In [ ]:
# log funding variant
import plotly.express as px
import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn.metrics import r2_score

df = pd.read_csv("funding_only.csv")

# Create log-transformed column for proper log-linear regression
df['log_funding'] = np.log10(df['Amount Raised'])

fig = px.scatter(
    df,
    x="log_funding",
    y="plr_norm",
    hover_name="Company",
    trendline="ols",
    title="Loudness vs Funding",
)

# Trendline style + hover (combined into single update)
fig.update_traces(
    selector=dict(mode="lines"),   # selects only the trendline traces
    line=dict(color="black", width=2, dash="dot"),  # customize style
    showlegend=True,  # make them appear in legend
    name="OLS Fit",
    hoverinfo="skip",  # completely disable hover for trendlines
    hovertemplate=None  # also set hovertemplate to None
)

fig.update_layout(
    template="simple_white",
    font=dict(family="Times New Roman", size=12, color="black"),
    xaxis_title="Funding Amount ($M)",
    yaxis_title="Loudness (PLR)",
    showlegend=False,
    width=700,
    height=500,
    margin=dict(l=40, r=20, t=40, b=40),
    xaxis=dict(
        tickmode='array',
        tickvals=[np.log10(3), np.log10(10), np.log10(30), np.log10(100), np.log10(300)],
        ticktext=['3', '10', '30', '100', '300'],
        showgrid=False,
        showline=True,
        zeroline=False,
    ),
    yaxis=dict(
        range=[0, 1],
        tickmode='array',
        tickvals=[0, .2, .4, .6, .8, 1],
        ticktext=['0', '.2', '.4', '.6', '.8', '1'],
        showgrid=False,
        showline=True,
        zeroline=False,
        
    ),
    title_font=dict(size=18),
    xaxis_tickfont=dict(size=14),
    yaxis_tickfont=dict(size=14),
)

# Scatter style + hover
fig.update_traces(
    selector=dict(mode="markers"),
    marker=dict(color="#016144",size=5, opacity=1, line=dict(width=0)),
    hovertemplate=
    "%{hovertext}<br>" +
    "Funding: $%{customdata:.0f}M<br>" +
    "PLR: %{y:.2f}<extra></extra>",
    customdata=df['Amount Raised']  # Use original funding values for hover
)

# r2 annotation calculation using log10 scaled funding
X_log = df['log_funding']  # Use the pre-calculated log column
y = df['plr_norm']
X_log = sm.add_constant(X_log)  # Add intercept
model = sm.OLS(y, X_log).fit()
r2 = model.rsquared

# r2 annotation
fig.add_annotation(
    xref="paper", 
    yref="paper",
    x=1.01, 
    y=0.8,
    text=f"R² = {r2:.3f}",
    showarrow=False,
    font=dict(
        size=12, 
        family="Times New Roman", 
        color="black"
    ),
)

# global design mechanics
fig.update_layout(
    dragmode=False,                 # disables drag-to-zoom, pan, lasso, etc.
    xaxis_fixedrange=True,          # lock x-axis range
    yaxis_fixedrange=True           # lock y-axis range
)

# modebar config
config = {
    "displaylogo": False,   # Plotly logo
    "modeBarButtonsToRemove": [
        "zoom2d",           # Zoom
        "pan2d",            # Pan
        "select2d",         # Box select
        "lasso2d",          # Lasso select
        "zoomIn2d",         # Zoom in
        "zoomOut2d",        # Zoom out
        "autoScale2d",      # Autoscale
        "resetScale2d",     # Reset axes
        "hoverClosestCartesian", # Closest point hover
        "hoverCompareCartesian", # Compare data hover
        "toggleSpikelines", # Spikelines
        "toImage",          # Download as PNG
        "sendDataToCloud",  # Deprecated cloud save
        "resetViews",       # Reset 3D views
        "orbitRotation",    # 3D orbit
        "tableRotation",    # 3D table rotation
        "zoom3d", "pan3d", "resetCameraDefault3d",
        "resetCameraLastSave3d", "hoverClosest3d",
        "toggleHover",      # Turn hover on/off
        "toggleHoverClosest", # (legacy alias)
        "toggleHoverCompare",
        "resetViews", "resetGeo", "resetViewMapbox"
    ],
    "displayModeBar": True,   # Always show
    # "displayModeBar": False # Never show
}

fig.show(config=config)


In [59]:
# John sitting vs standing loudness boxplot
import plotly.express as px
import pandas as pd

# Load data and prepare John's position data
df = pd.read_csv("funding_only.csv")
john_df = df[df["Gong Master"] == "John"].copy()
john_df["Position"] = john_df["john_sitting"].apply(lambda x: "Sitting" if x == 1 else "Standing")

# Create swarm chart
fig = px.strip(
    john_df,
    x="Position",
    y="plr_norm",
    color="Position",
    hover_name="Company",
    category_orders={"Position": ["Standing", "Sitting"]},
    stripmode="overlay"
)

# Define colors and prepare position data once
colors = {"Standing": "black", "Sitting": "black"}
position_data = {pos: john_df[john_df["Position"] == pos] for pos in ["Standing", "Sitting"]}

# Style traces and add median lines in single loop
for i, (position, trace) in enumerate(zip(["Standing", "Sitting"], fig.data)):
    data_subset = position_data[position]
    median = data_subset["plr_norm"].median()
    
    # Update trace styling
    trace.update(
        marker=dict(
            color=colors[position],
            size=8,
            opacity=1,
            line=dict(width=0.5, color="black")
        ),
        hovertemplate="%{hovertext}<br>Funding: $%{x:.0f}M<br>PLR: %{y:.2f}<extra></extra>",
        hoverlabel=dict(
            font=dict(family="Times New Roman", color="black"),
            bgcolor="white",
            bordercolor="black"
    )
    )
    

    # Add median line and label
    fig.add_shape(
        type="line",
        x0=i-0.4, x1=i+0.4,
        y0=median, y1=median,
        line=dict(color="black", width=2, dash="dash")
    )
    
    fig.add_annotation(
        x=i + 0.45, y=median,
        text=f"{median:.2f}",
        showarrow=False,
        font=dict(family="Times New Roman", size=10, color="black"),
        xanchor="left"
    )

# Complete layout configuration
fig.update_layout(
    template="simple_white",
    font=dict(family="Times New Roman", size=12, color="black"),
    xaxis_title="",
    yaxis_title="Loudness (PLR)",
    showlegend=False,
    width=600,
    height=600,
    margin=dict(l=80, r=80, t=40, b=60),
    xaxis=dict(
        showgrid=False,
        showline=False,
        zeroline=False,
        tickfont=dict(size=14)
    ),
    yaxis=dict(
        range=[.2, 1],
        tickmode='array',
        tickvals=[0.2, 0.4, 0.6, 0.8, 1.0],
        showgrid=False,
        showline=True,
        linecolor="white",
        linewidth=1,
        zeroline=False,
        tickfont=dict(size=12)
    ),
    plot_bgcolor="white",
    paper_bgcolor="white"
)

# Show chart with hover enabled
fig.show(config={"displaylogo": False, "displayModeBar": False, "staticPlot": False})

fig.write_html("chart_7_john_sitting_plr.html")


In [52]:
# John stands for size
import plotly.express as px
import pandas as pd

# Load data and calculate sitting percentages by funding threshold
df = pd.read_csv("funding_only.csv")
john_df = df[df["Gong Master"] == "John"]

# Calculate sitting percentages for funding thresholds
thresholds = [0, 10, 20, 30, 40]
threshold_labels = ["All Rounds", "Above $10M", "Above $20M", "Above $30M", "Above $40M"]

sitting_percentages = [
    john_df[john_df["Amount Raised"] > threshold]["john_sitting"].mean() 
    if len(john_df[john_df["Amount Raised"] > threshold]) > 0 else 0
    for threshold in thresholds
]

# Create bar chart
fig = px.bar(x=threshold_labels, y=sitting_percentages, title="")

# Minimalist styling
fig.update_layout(
    template="simple_white",
    font=dict(family="Times New Roman", size=14, color="black"),
    xaxis_title="",
    yaxis_title="",
    showlegend=False,
    width=500,
    height=400,
    margin=dict(l=40, r=20, t=40, b=60),
    xaxis=dict(
        showgrid=False,
        showline=True,
        linecolor="black",
        linewidth=1,
        zeroline=False,
        tickfont=dict(size=14),
        tickangle=-45
    ),
    yaxis=dict(
        range=[0, 0.4],
        tickmode='array',
        tickvals=[0, 0.1, 0.2, 0.3, 0.4],
        ticktext=['0.0%', '10.0%', '20.0%', '30.0%', '40.0%'],
        showgrid=False,
        showline=True,
        linecolor="black",
        linewidth=1,
        zeroline=False,
        tickfont=dict(size=14)
    ),
    plot_bgcolor="white",
    paper_bgcolor="white"
)

# Bar styling
fig.update_traces(
    marker=dict(
        color="#016144",
        line=dict(width=1, color="black"),
        opacity=0.8
    ),
    hoverinfo="none"
)

# Add percentage labels
for i, percentage in enumerate(sitting_percentages):
    fig.add_annotation(
        x=i, y=percentage + 0.005,
        text=f"{percentage:.1%}",
        showarrow=False,
        font=dict(family="Times New Roman", size=12, color="black"),
        xanchor="center",
        yanchor="bottom"
    )

# Show chart
fig.show(config={"displaylogo": False, "displayModeBar": False, "staticPlot": True})

fig.write_html("chart_6_john_sitting_by_amount_raised.html")


In [122]:
# the results
import plotly.express as px
import pandas as pd
import statsmodels.api as sm

# Load data and create scatter plot with color coding and trendlines
df = pd.read_csv("funding_only.csv")
fig = px.scatter(
    df,
    x="Amount Raised",
    y="plr_norm",
    color="Gong Master",
    hover_name="Company",
    custom_data=["timestamped_link"],  # Add URL data for click functionality
    trendline="ols",
    color_discrete_map={"John": "#016144", "Jordi": "#FF6B35"},
    title=""
)

# Style trendlines for each gong master
fig.update_traces(
    selector=dict(mode="lines", name="John"),
    line=dict(color="#016144", width=2, dash="dot"),
    hoverinfo="skip",
    hovertemplate=None,
    showlegend=False
)

fig.update_traces(
    selector=dict(mode="lines", name="Jordi"),
    line=dict(color="#d6584a", width=2, dash="dot"),
    hoverinfo="skip",
    hovertemplate=None,
    showlegend=False
)

# Style markers for each gong master
fig.update_traces(
    selector=dict(mode="markers", name="John"),
    marker=dict(color="#4C6A91", size=7, opacity=1, line=dict(width=0)),
    hovertemplate="<span style='font-weight: 600; font-family: gill sans;'>%{hovertext}</span><br>Funding: $%{x:.0f}M<br>PLR: %{y:.2f}<br>Gong Master: John<extra></extra>",
    hoverlabel=dict(
        font=dict(family="gill sans", color="black"),
        bgcolor="white",
        bordercolor="black"
    )
)

fig.update_traces(
    selector=dict(mode="markers", name="Jordi"),
    marker=dict(color="#D6584A", size=7, opacity=1, line=dict(width=0)),
    hovertemplate="<span style='font-weight: 600; font-family: gill sans;'>%{hovertext}</span><br>Funding: $%{x:.0f}M<br>PLR: %{y:.2f}<br>Gong Master: Jordi<extra></extra>",
    hoverlabel=dict(
        font=dict(family="gill sans", color="black"),
        bgcolor="white",
        bordercolor="black"
    )
)

# Calculate R² for each gong master
john_data = df[df['Gong Master'] == 'John']
jordi_data = df[df['Gong Master'] == 'Jordi']

X_john = sm.add_constant(john_data['Amount Raised'])
r2_john = sm.OLS(john_data['plr_norm'], X_john).fit().rsquared

X_jordi = sm.add_constant(jordi_data['Amount Raised'])
r2_jordi = sm.OLS(jordi_data['plr_norm'], X_jordi).fit().rsquared

# Complete layout configuration
fig.update_layout(
    template="simple_white",
    font=dict(family="monotype bembo", size=14, color="black"),
    xaxis_title="Funding Amount ($M)",
    yaxis_title="Loudness (PLR)",
    showlegend=False,
    legend=dict(
        x=0.02,
        y=0.98,
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1,
        font=dict(family="gill sans", size=12)
    ),
    width=809,
    height=500,
    margin=dict(l=40, r=20, t=40, b=40),
    title_font=dict(size=18),
    dragmode=False,
    xaxis=dict(
        range=[0, 300],
        showgrid=False,
        showline=True,
        zeroline=False,
        tickfont=dict(size=14),
        fixedrange=True
    ),
    yaxis=dict(
        range=[.2, 1],
        tickmode='array',
        tickvals=[0.2, 0.4, 0.6, 0.8, 1],
        ticktext=['.2', '.4', '.6', '.8', '1'],
        showgrid=False,
        showline=True,
        zeroline=False,
        tickfont=dict(size=14),
        fixedrange=True
    )
)

# Add R² annotations for each gong master
fig.add_annotation(
    xref="paper", yref="paper",
    x=0.96, y=0.9,
    text=f"John R² = {r2_john:.3f}",
    showarrow=False,
    font=dict(size=12, family="gill sans", color="#016144")
)

fig.add_annotation(
    xref="paper", yref="paper",
    x=0.96, y=0.84,
    text=f"Jordi R² = {r2_jordi:.3f}",
    showarrow=False,
    font=dict(size=12, family="gill sans", color="#d6584a")
)

# Show chart with click functionality enabled
config = {
    "displaylogo": False, 
    "displayModeBar": False
}
fig.show(config=config)

# Write HTML with embedded JavaScript for click handling
html_string = fig.to_html(config=config, include_plotlyjs=True)

# Add JavaScript for click handling
click_js = '''
<script>
document.addEventListener('DOMContentLoaded', function() {
    setTimeout(function() {
        var plotDiv = document.querySelector('.js-plotly-plot');
        if (plotDiv) {
            plotDiv.on('plotly_click', function(data) {
                if (data.points && data.points[0] && data.points[0].customdata) {
                    var url = data.points[0].customdata[0];
                    if (url) {
                        window.open(url, '_blank');
                    }
                }
            });
        }
    }, 100);
});
</script>
'''

# Insert JavaScript before closing body tag
html_with_clicks = html_string.replace('</body>', click_js + '</body>')

# Save the interactive HTML file
with open("chart_2_hosts_split.html", "w") as f:
    f.write(html_with_clicks)

